In [46]:
import os
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import time # this to add delay between list call

In [2]:
# get date today in mm-dd-yyyy format
today = datetime.today().strftime('%m-%d-%Y')

In [4]:
content = json.dumps(json_data, indent = 4, sort_keys=True)
json_content = json.loads(content)
stock_data = json_content['records']
stocks = []
for item in stock_data:
    stocks.append([
        item['companyId'],
        item['companyName'],
        item['indexName'],
        item['listingDate'],
        item['securityName'],
        item['securitySymbol'],
        item['securitySymbolId'],
        item['securityType'],
        item['subsectorName']
    ])

stocks

In [51]:
for stock in stocks:
    print(stock[0],stock[5],stock[6]) # company_id

29
2GO
146
626
8990P
637
626
HOUSE
563
13
BRN
234
14
ANS
169
114
ABS
123
15
ABSP
259
233
ACEPH
140
653
ACEX
599
177
APC
182
56
ATN
176
56
ATNB
175
174
ABA
173
16
AEV
183
609
AP
532
33
AR
321
48
ACE
370
619
ANI
547
674
HOME
647
212
AGI
179
602
FOOD
524
121
ACR
170
612
ALHI
536
52
APO
171
178
APX
318
638
APL
581
38
ARA
294
172
ALCO
316
172
ALCPB
630
172
ALCPC
644
55
AAA
347
641
AUB
584
176
ABG
350
53
ATI
141
34
AT
320
19
AB
181
673
AXLM
646
57
AC
180
57
ACPA
546
57
ACPB1
523
57
APB2R
651
180
ALI
293
26
ALLHC
202
31
BLFI
121
260
BDO
468
62
BH
358
234
BPI
101
60
BSC
188
21
BEL
252
108
BC
353
108
BCB
322
108
BCP
354
9
BCOR
391
49
BLOOM
233
181
BMM
359
63
BHI
187
66
BKR
116
601
COL
522
624
CEB
560
110
CHI
298
668
CLI
635
182
CPV
296
182
CPVB
368
662
CHP
625
183
CAT
366
223
CEU
291
652
CNPF
597
621
CPM
549
189
CPG
219
189
CPGP
652
669
C
636
22
CIP
367
184
CHIB
104
630
TECB2
638
630
TECH
569
209
LAND
308
39
CDC
297
228
CSB
457
637
COAL
580
648
CIC
591
213
CA
361
213
CAB
362
50
COSCO
178
657
CR

In [9]:
stocks_df = pd.DataFrame(stocks)
stocks_df.columns = ['companyId','companyName','indexName','listingDate','securityName','securitySymbol','securitySymbolId','securityType','subsectorName']
stocks_df.head()
stocks_df.dtypes
stocks_df.rename(columns={
    'companyId': 'company_id',
    'companyName': 'company_name',
    'indexName': 'index_name',
    'listingDate': 'listing_date',
    'securityName': 'security_name',
    'securitySymbol': 'security_symbol',
    'securitySymbolId': 'security_symbol_id',
    'securityType': 'security_type',
    'subsectorName': 'subsector_name'
},inplace=True)
stocks_df['listing_date'] = stocks_df['listing_date'].apply(pd.to_datetime)
stocks_df.index = stocks_df['security_symbol_id']
stocks_df.drop('index_name', axis=1, inplace=True)
stocks_df.loc['224']

In [17]:
# use security symbol id as security id
url = 'https://edge.pse.com.ph/common/DisclosureCht.ax'

security_data = requests.post(
    url,
    headers={'Content-Type':'application/json','Referer':'https://edge.pse.com.ph/companyPage/stockData.do?cmpy_id=160'},
    json={'cmpy_id':160,'security_id':224,'endDate':'05-30-2020','startDate':'01-01-2009'}).json()

In [30]:
url = 'https://edge.pse.com.ph/common/DisclosureCht.ax'
def get_stock_data(companyName=None, companyId=None, securityId=None, endDate=None, startDate=None):
    # define referer in the headers
    referer = 'https://edge.pse.com.ph/companyPage/stockData.do?cmpy_id=%d' % securityId
    # define data to be sent to server
    json = {
        'cmpy_id': companyId,
        'security_id': securityId,
        'endDate': endDate,
        'startDate': startDate
    }
    # define headers
    headers = {
        'Content-Type':'application/json',
        'Referer': referer
    }
    # call request
    security_data = requests.post(url, headers=headers, json=json).json()
    # return json data
    return security_data

In [31]:
result = get_stock_data(160,224,'05-29-2020','01-01-2019')

In [32]:
print(result)

{'chartData': [{'OPEN': 1.34, 'VALUE': 25280.0, 'CLOSE': 1.33, 'CHART_DATE': 'Jan 02, 2019 00:00:00', 'HIGH': 1.34, 'LOW': 1.33}, {'OPEN': 1.32, 'VALUE': 13200.0, 'CLOSE': 1.32, 'CHART_DATE': 'Jan 03, 2019 00:00:00', 'HIGH': 1.32, 'LOW': 1.32}, {'OPEN': 1.31, 'VALUE': 185080.0, 'CLOSE': 1.33, 'CHART_DATE': 'Jan 04, 2019 00:00:00', 'HIGH': 1.34, 'LOW': 1.29}, {'OPEN': 1.31, 'VALUE': 260440.0, 'CLOSE': 1.34, 'CHART_DATE': 'Jan 07, 2019 00:00:00', 'HIGH': 1.34, 'LOW': 1.29}, {'OPEN': 1.34, 'VALUE': 1455530.0, 'CLOSE': 1.4, 'CHART_DATE': 'Jan 08, 2019 00:00:00', 'HIGH': 1.45, 'LOW': 1.34}, {'OPEN': 1.4, 'VALUE': 257840.0, 'CLOSE': 1.39, 'CHART_DATE': 'Jan 09, 2019 00:00:00', 'HIGH': 1.4, 'LOW': 1.38}, {'OPEN': 1.39, 'VALUE': 381900.0, 'CLOSE': 1.4, 'CHART_DATE': 'Jan 10, 2019 00:00:00', 'HIGH': 1.4, 'LOW': 1.35}, {'OPEN': 1.4, 'VALUE': 760740.0, 'CLOSE': 1.46, 'CHART_DATE': 'Jan 11, 2019 00:00:00', 'HIGH': 1.48, 'LOW': 1.39}, {'OPEN': 1.44, 'VALUE': 154290.0, 'CLOSE': 1.42, 'CHART_DATE': '

In [33]:
type(result)

dict

In [36]:
chart_data = result['chartData']
type(chart_data)

list

In [38]:
chart_data = result['chartData']
chart_data_df = pd.DataFrame(chart_data)
chart_data_df.columns = ['open','value','close','chart_date','high','low']
chart_data_df = chart_data_df[['chart_date','open','high','low','close','value']]
chart_data_df.to_csv('SGI.csv',index=False)

chart_data_df.sample(10)

,open,value,close,chart_date,high,low
204,1.23,76280.0,1.23,"Dec 10, 2019 00:00:00",1.24,1.23
252,0.91,160480.0,0.95,"Mar 10, 2020 00:00:00",0.95,0.90
209,1.22,203740.0,1.22,"Dec 17, 2019 00:00:00",1.22,1.22
175,1.26,13860.0,1.26,"Oct 21, 2019 00:00:00",1.26,1.26
266,0.83,1418510.0,0.84,"Apr 03, 2020 00:00:00",0.84,0.83
164,1.29,330910.0,1.29,"Sep 25, 2019 00:00:00",1.29,1.27
94,1.36,17690.0,1.37,"Jun 07, 2019 00:00:00",1.37,1.36
181,1.27,5080.0,1.27,"Oct 31, 2019 00:00:00",1.27,1.27
44,1.35,21540.0,1.36,"Mar 13, 2019 00:00:00",1.36,1.34
113,1.40,614660.0,1.38,"Jul 08, 2019 00:00:00",1.42,1.38


In [3]:

# data = json.dumps(json_data,sort_keys=False)

{'count': 328, 'totalCount': 328, 'records': [{'securityStatus': 'O', 'listingDate': '1995-05-15 00:00:00.0', 'securityType': 'S', 'subsectorName': '13', 'indexName': '', 'securitySymbol': '2GO', 'securityName': '2GO GROUP, INC.', 'companyId': '29', 'companyName': '2GO Group, Inc.', 'securitySymbolId': '146'}, {'securityStatus': 'O', 'listingDate': '2017-12-01 00:00:00.0', 'securityType': 'P', 'subsectorName': '9', 'indexName': '', 'securitySymbol': '8990P', 'securityName': '8990 Holdings, Inc. Series A Perpetual Preferred Shares', 'companyId': '626', 'companyName': '8990 Holdings, Inc.', 'securitySymbolId': '637'}, {'securityStatus': 'O', 'listingDate': '2010-12-09 00:00:00.0', 'securityType': 'S', 'subsectorName': '9', 'indexName': '', 'securitySymbol': 'HOUSE', 'securityName': '8990 HOLDINGS, INC.', 'companyId': '626', 'companyName': '8990 Holdings, Inc.', 'securitySymbolId': '563'}, {'securityStatus': 'O', 'listingDate': '1994-02-08 00:00:00.0', 'securityType': 'S', 'subsectorName'